In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.feature_selection import VarianceThreshold,SelectFromModel
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, confusion_matrix


In [3]:
df=pd.read_csv('E:\\NAAMII\\Machine_learning\\dataset\\train_set.csv')

In [11]:
target = df['CLASS']

In [4]:
features = df.drop(['CLASS', 'ID'], axis=1, errors='ignore')


In [6]:
# Check for infinite and extreme values first
print(f"\nData Quality Check:")
inf_count = np.isinf(features.values).sum()
nan_count = np.isnan(features.values).sum()
print(f"Infinite values: {inf_count}")
print(f"NaN values: {nan_count}")


Data Quality Check:
Infinite values: 4
NaN values: 2668


In [7]:
# Replace infinite values with NaN for statistics calculation
features_clean = features.replace([np.inf, -np.inf], np.nan)
features_clean.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,564.936250,...,382.968383,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710
1,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,31.291507,...,452.986164,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548
2,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,11.965643,...,419.781765,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704
3,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,8.966286,...,439.023968,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803
4,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,34.898671,...,485.209184,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608


In [8]:
features_clean.dropna(axis=1, thresh=0.9 * len(features_clean))

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,564.936250,...,382.968383,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710
1,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,31.291507,...,452.986164,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548
2,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,11.965643,...,419.781765,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704
3,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,8.966286,...,439.023968,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803
4,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,34.898671,...,485.209184,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,46787.916667,47002.0,18052.070312,0.385828,0.010883,0.042086,2.874885,0.347840,1.874885,23.499143,...,466.276055,466.276055,6064.0,1.0,585.547823,0.096561,0.0,28.787507,3.894684,0.096561
311,8420.354167,8493.0,4292.039795,0.510004,0.016911,0.101797,2.145061,0.466555,1.145061,25.293867,...,383.044821,383.044821,987.0,1.0,69.155790,0.070369,0.0,29.625473,4.098452,0.070369
312,37262.750000,37407.0,13950.793945,0.374390,0.012759,0.057837,2.585819,0.386725,1.585819,37.474634,...,469.005263,469.005263,4940.0,1.0,505.566802,0.102341,0.0,26.865256,3.815115,0.102341
313,25081.833333,25251.0,11689.275391,0.466045,0.011197,0.044546,2.820962,0.354489,1.820962,18.321132,...,403.597826,403.597826,2944.0,1.0,215.172554,0.073089,0.0,29.180584,4.055504,0.073089


In [9]:
# Impute remaining NaNs with column mean
features_clean.fillna(features_clean.mean(), inplace=True)


In [10]:
# 3. REMOVE ZERO-VARIANCE FEATURES
var_thresh = VarianceThreshold(threshold=0.0)
X_var = var_thresh.fit_transform(features_clean)

In [15]:
# REMOVE HIGHLY CORRELATED FEATURES
def remove_high_correlation_features(X, threshold=0.95):
    corr_matrix = pd.DataFrame(X).corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return pd.DataFrame(X).drop(columns=to_drop, axis=1)

X_clean = remove_high_correlation_features(X_var, threshold=0.95)

In [16]:
# Helper function to calculate all metrics
def evaluate(y_true, y_pred, y_proba):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Specificity': specificity,
        'AUROC': roc_auc_score(y_true, y_proba)
    }

# Define models
models = {
    "RandomForest": RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42,max_depth=None,min_samples_leaf=2, min_samples_split=5),
    "XGBoost": XGBClassifier(scale_pos_weight=191/124, eval_metric='logloss', random_state=42,learing_rate=0.2, n_estimators=200, max_depth=3, subsample=0.8, colsample_bytree=0.8, use_label_encoder=False, verbosity=0),
}

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {model_name: [] for model_name in models}

# Loop through models and perform cross-validation
for model_name, model in models.items():
    print(f"\nTraining {model_name}")
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_clean, target)):
        X_train, X_val = X_clean[train_idx], X_clean[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]
        
        metrics = evaluate(y_val, y_pred, y_proba)
        results[model_name].append(metrics)
        print(f" Fold {fold+1}: " + ", ".join([f"{k}: {v:.4f}" for k, v in metrics.items()]))

# Average results
print("\n📊 Average Cross-Validation Results:")
for model_name, folds in results.items():
    avg = {k: np.mean([fold[k] for fold in folds]) for k in folds[0]}
    print(f" {model_name}: " + ", ".join([f"{k}: {v:.4f}" for k, v in avg.items()]))



Training RandomForest


KeyError: '[1, 5, 6, 7, 8, 11, 20, 24, 25, 28, 29, 30, 33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 53, 62, 67, 68, 69, 70, 74, 76, 77, 78, 79, 81, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 97, 98, 100, 103, 104, 106, 107, 108, 121, 122, 123, 125, 126, 128, 129, 131, 135, 137, 138, 139, 142, 144, 145, 146, 147, 148, 150, 151, 152, 154, 155, 156, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 201, 202, 203, 204, 205, 206, 208, 209, 211, 212, 214, 216, 217, 220, 224, 225, 226, 227, 228, 230, 231, 233, 234, 235, 237, 240, 241, 242, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 265, 266, 267, 270, 271, 272, 273, 274, 276, 277, 278, 279, 280, 281, 282, 283, 285, 286, 287, 288, 289, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 309, 310, 311, 312, 313, 314] not in index'